Objective: Identify MitoProximal genes using hypergeometric means to determine the overlap in STRING networks and the MitoCarta 3.0 gene list. Ensembl protein IDs are used as the input into the hypergeometric test.

In [ ]:
#Setting up the environment- loading packages needed
import re, pandas as pd, math as m
from scipy.stats import hypergeom
import os

In [ ]:
###########################
# Variable declaration
###########################

# define your folder to the Mitocarta csv file as a string
mitocarta_dir = ''

# define your Mitocarta csv file listing the Ensembl Protein ID's for all genes as a string ('File')
experimental_geneset = mitocarta_dir + ''

#directory containing each STRING target gene with its neighbors as a string
stringlists_dir = ''

#Define the established genesets (e.g., STRING targets with their clouds)
established_genesets  = os.listdir(stringlists_dir)

M_all_genes = 18872 # number of all human genes having neighbors in String  
# n_target_neighbors = number of gene nearest neighbors from String DB. Cloud of neighbors around each individual gene. Will be generated for each gene on the fly
N_sig_genes =  1135 # number of genes in Mitocarta. Note: a long non-coding RNA is including in the list of MitoCarta 3.0 genes, which was removed as it is not be in the STRING protein networks 
# x_sig_neighbors = the overlap (number of Mitocarta_plus genes that are also the target neighbors). Will be generated for each gene on the fly

In [ ]:
#Calculating the number of neighbors for each target gene by processing the STRING files we created for each neighbor
def read_processed_target_neighbors(established_geneset):    #established_geneset in this case is each STRING text file 
    neighbor_dict = {}
    num_neighbors = 0
    target_neighbors_file = stringlists_dir + established_geneset
    data = pd.read_table(target_neighbors_file)
    
    num_dowstream = 0
    num_upstream = 0

    for row in data.values:    
        ensembl = row[0].upper().strip()
        hugo = row[1].upper().strip()
        num_neighbors += 1
        neighbor_dict[ensembl] = established_geneset 
        
    return(num_neighbors, neighbor_dict)

In [ ]:
#Creating a list of the ensembl protein id's for the MitoCarta genes
mito_genes = []
with open(experimental_geneset) as f:
    next(f) #skipping first row since it's a header
    for line in f:
        row = line.split(',')
        hugo = row[0].upper()
        ensembl = row[1].strip(' ')
        if ensembl == "":
            pass
        elif ensembl == "-":
            pass
        else:
            #print(ensembl)
            mito_genes.append(ensembl)  

In [ ]:
#Counting the number of times the genes in the experimental geneset (MitoCarta) overlap with the established geneset (STRING neighbors of the current gene)
def overlap_with_established_geneset(experimental_geneset, established_geneset): #experimental_geneset is the MitoCarta file
    
    x_sig_neighbors = 0
    
    mito_genes = []
    with open(experimental_geneset) as f:
        next(f) #skipping first row since it's a header
        for line in f:
            row = line.split(',')
            hugo = row[0].upper()
            ensembl = row[1].strip(' ')
            if ensembl == "":
                pass
            elif ensembl == "-":
                pass
            else:
                #print(ensembl)
                mito_genes.append(ensembl)  
    
    for ensembl in mito_genes:
        if ensembl in neighbor_dict:
            x_sig_neighbors += 1
    
    return(x_sig_neighbors)

In [ ]:
#############################################
# Hypergeometric test
#############################################
#note that the established_geneset is the cloud of neighbors from STRING
def hypergeom_test(established_geneset):
    
    pval = hypergeom.sf(x_sig_neighbors-1, M_all_genes, n_target_neighbors, N_sig_genes)
    print (established_geneset.replace('_string_neighbors.txt','').replace('/restricted/project/icamp/mito-nDNA-genes/string_lists/', ''), x_sig_neighbors, M_all_genes, N_sig_genes, n_target_neighbors, pval, sep="\t")
    print (established_geneset.replace('_string_neighbors.txt','').replace('/restricted/project/icamp/mito-nDNA-genes/string_lists/', ''), x_sig_neighbors, M_all_genes, N_sig_genes, n_target_neighbors, pval, sep="\t", file = fout)

# open output file (better to give it a dynamic name based on experimental geneset)
fout = open("outfile.txt", 'w')
print ( '\n','\tgene', 'ovrlp', 'ngenes', 'siggns', 'neibrs', 'pval', sep="\t")
print ( '\n','\tgene', 'ovrlp', 'ngenes', 'siggns', 'neibrs', 'pval', sep="\t", file = fout)

x_sig_neighbors = 0

#this for-loop is to compare each "cloud" of neighbors with Mitocarta or another genelist 
for est_geneset in established_genesets:
    
    n_target_neighbors, neighbor_dict = read_processed_target_neighbors(est_geneset) 
    
    x_sig_neighbors = overlap_with_established_geneset(experimental_geneset, est_geneset)
    
    hypergeom_test(est_geneset)

fout.close()